#### 부스팅 기법
##### ada부스팅은 과적합을 줄일 수 있음(에러값들 모아두는 함수구하는 함수 손실함수라함)
##### 

##### 그래디언(기울기)부스팅은 에러값들 다시 조정하면서 기울기값을 계속 줄여줌
##### class sklearn.ensemble.GradientBoostingClassifier(*, loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
###### subsample은 0과 1사이의 값으로 해야 작으므로 그사이값으로 지정해줘야함



###### XG부스팅은 시간도 빨라지고, 예측도 좋아짐 그래디언 부스팅과 차이점은 크게 크게 나누고, 정규화까지 시켜주는 차이점이 있음


### 1.각각의 함수를 만들어 전처리와 변수저장을 다 해줌

In [2]:

import pandas as pd



def get_new_feature_name_df(old_feature_name_df): # 인덱스값을 제거해서 전처리하는 과정
    feature_dup_df=pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    feature_dup_df= feature_dup_df.reset_index()
    new_feature_name_df=pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')
    new_feature_name_df['column_name']=new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                            if x[1]>0 else x[0], axis=1)
    new_feature_name_df=new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

def get_human_dataset( ): # 파일들 변수에 저장해주는 함수
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./drive/MyDrive/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name']) #CSV불러오는 
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./drive/MyDrive/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./drive/MyDrive/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./drive/MyDrive/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./drive/MyDrive/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
import get_human_dataset_copy as gh #모듈 써준 거
warnings.filterwarnings('ignore') 
from sklearn.metrics import accuracy_score



### 2.함수 불러와서 작성

In [3]:
X_train,X_test,y_train,y_test =gh.get_human_dataset() #함수 불러와서 작성

In [4]:
start_time=time.time() #시간 작성 

gb_clf = GradientBoostingClassifier(random_state=0) #gb부스팅 쓰기 
gb_clf.fit(X_train,y_train) #gb부스팅 훈련
gb_pred =gb_clf.predict(X_test) #예상값 가져오기
gb_accuracy = accuracy_score(y_test,gb_pred) #정확도 측정

print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))
print('GBM 수행시간:{0:.1f}초'.format(time.time()-start_time))

### 3. 그리드서치CV 를 통해서 하이퍼 파라미터에서 최적화를 찾음

In [ ]:
from sklearn.model_selection import GridSearchCV

params={
    'n_estimators':[100,500],
    'learning_rate':[0.05,0.1]
}

grid_cv=GridSearchCV(gb_clf,param_grid=params , cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파리미터: /n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

NameError: name 'gb_clf' is not defined